# 서울시 편의점 매출 결정요인 분석
## 04. 회귀분석

---

### 이 노트북의 목표
EDA와 시각화에서 확인한 가설을 통계적으로 검증

### 검증할 가설
- **H1**: 유동인구가 많을수록 매출이 높다
- **H2**: 점포수(밀집도)가 높을수록 매출이 높다
- **H3**: 상권유형에 따라 매출 차이가 있다

### 모델 설계
```
매출 = β₀ + β₁(유동인구) + β₂(점포수) + β₃(발달상권) + β₄(전통시장) + β₅(관광특구) + β₆(미분류) + ε
기준 범주: 골목상권
```

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

df = pd.read_csv('./분석데이터.csv', encoding='utf-8-sig')
print(f"데이터: {len(df):,}건")

데이터: 6,097건


## 1. 변수 준비

In [2]:
# 더미변수 생성 (기준: 골목상권)
df['발달상권'] = (df['주요_상권유형'] == '발달상권').astype(int)
df['전통시장'] = (df['주요_상권유형'] == '전통시장').astype(int)
df['관광특구'] = (df['주요_상권유형'] == '관광특구').astype(int)
df['미분류'] = (df['주요_상권유형'] == '미분류').astype(int)

y = df['당월_매출_금액']
X_cols = ['점포_수', '총_유동인구_수', '발달상권', '전통시장', '관광특구', '미분류']
X = df[X_cols]
X_const = sm.add_constant(X)

print(f"종속변수: 당월_매출_금액")
print(f"독립변수: {X_cols}")
print(f"기준범주: 골목상권")

종속변수: 당월_매출_금액
독립변수: ['점포_수', '총_유동인구_수', '발달상권', '전통시장', '관광특구', '미분류']
기준범주: 골목상권


## 2. 다중공선성 점검 (VIF)

In [3]:
print("[VIF 분석] 기준: <5 양호, 5~10 주의, >10 심각")
print("-" * 40)

max_vif = 0
for i, col in enumerate(X_cols):
    vif = variance_inflation_factor(X_const.values, i+1)
    max_vif = max(max_vif, vif)
    status = '양호' if vif < 5 else ('주의' if vif < 10 else '심각')
    print(f"  {col}: {vif:.2f} ({status})")

print(f"\n최대 VIF: {max_vif:.2f} → {'문제없음' if max_vif < 5 else '주의필요'}")

[VIF 분석] 기준: <5 양호, 5~10 주의, >10 심각
----------------------------------------
  점포_수: 1.47 (양호)
  총_유동인구_수: 1.41 (양호)
  발달상권: 1.10 (양호)
  전통시장: 1.01 (양호)
  관광특구: 1.04 (양호)
  미분류: 1.06 (양호)

최대 VIF: 1.47 → 문제없음


## 3. 회귀모델 적합

In [4]:
model = sm.OLS(y, X_const).fit()

print("[모델 적합도]")
print(f"  R²: {model.rsquared:.4f} (매출 변동의 {model.rsquared*100:.1f}% 설명)")
print(f"  Adj R²: {model.rsquared_adj:.4f}")
print(f"  F: {model.fvalue:.2f}, p < 0.001")

[모델 적합도]
  R²: 0.6881 (매출 변동의 68.8% 설명)
  Adj R²: 0.6878
  F: 2238.95, p < 0.001


In [5]:
print("\n[회귀계수]")
print("=" * 65)
print(f"{'변수':<15} {'계수':>15} {'p-value':>12} {'유의':>6}")
print("-" * 65)

for var in model.params.index:
    coef = model.params[var]
    pval = model.pvalues[var]
    sig = '***' if pval < 0.001 else ('**' if pval < 0.01 else ('*' if pval < 0.05 else ''))
    print(f"{var:<15} {coef:>15,.0f} {pval:>12.2e} {sig:>6}")

print("-" * 65)
print("*** p<0.001, ** p<0.01, * p<0.05")


[회귀계수]
변수                           계수      p-value     유의
-----------------------------------------------------------------
const              -973,220,686     3.77e-74    ***
점포_수                399,192,392     0.00e+00    ***
총_유동인구_수                    186     7.47e-89    ***
발달상권              1,471,873,945     1.19e-39    ***
전통시장              1,194,138,984     5.99e-23    ***
관광특구              2,719,155,654     4.31e-44    ***
미분류                 439,417,000     2.08e-05    ***
-----------------------------------------------------------------
*** p<0.001, ** p<0.01, * p<0.05


## 4. 가설 검증

In [6]:
print("=" * 60)
print("가설 검증 결과")
print("=" * 60)

# H1
c1, p1 = model.params['총_유동인구_수'], model.pvalues['총_유동인구_수']
print(f"\n[H1] 유동인구 → 매출")
print(f"  계수: {c1:.2f}원/명")
print(f"  해석: 유동인구 100만명↑ → 매출 {c1*1e6/1e8:.2f}억원↑")
print(f"  p-value: {p1:.2e}")
print(f"  → H1 {'채택 ✓' if p1 < 0.05 and c1 > 0 else '기각'}")

# H2
c2, p2 = model.params['점포_수'], model.pvalues['점포_수']
print(f"\n[H2] 점포수 → 매출")
print(f"  계수: {c2:,.0f}원/개")
print(f"  해석: 점포 1개↑ → 매출 {c2/1e8:.2f}억원↑")
print(f"  p-value: {p2:.2e}")
print(f"  → H2 {'채택 ✓' if p2 < 0.05 and c2 > 0 else '기각'}")

# H3
print(f"\n[H3] 상권유형 → 매출 (기준: 골목상권)")
h3_sig = False
for v in ['발달상권', '전통시장', '관광특구', '미분류']:
    c, p = model.params[v], model.pvalues[v]
    sig = '***' if p < 0.001 else ('**' if p < 0.01 else ('*' if p < 0.05 else 'ns'))
    d = '↑' if c > 0 else '↓'
    print(f"  {v}: {abs(c)/1e8:.2f}억원{d} ({sig})")
    if p < 0.05: h3_sig = True
print(f"  → H3 {'채택 ✓' if h3_sig else '기각'}")

가설 검증 결과

[H1] 유동인구 → 매출
  계수: 185.94원/명
  해석: 유동인구 100만명↑ → 매출 1.86억원↑
  p-value: 7.47e-89
  → H1 채택 ✓

[H2] 점포수 → 매출
  계수: 399,192,392원/개
  해석: 점포 1개↑ → 매출 3.99억원↑
  p-value: 0.00e+00
  → H2 채택 ✓

[H3] 상권유형 → 매출 (기준: 골목상권)
  발달상권: 14.72억원↑ (***)
  전통시장: 11.94억원↑ (***)
  관광특구: 27.19억원↑ (***)
  미분류: 4.39억원↑ (***)
  → H3 채택 ✓


## 5. 모델 진단

In [7]:
residuals = model.resid

# 정규성
sample = np.random.choice(residuals, min(5000, len(residuals)), replace=False)
_, p_norm = stats.shapiro(sample)
print(f"[정규성] Shapiro p={p_norm:.2e} → {'불만족(대표본 OK)' if p_norm < 0.05 else '만족'}")

# 등분산성
_, p_het, _, _ = het_breuschpagan(residuals, X_const)
print(f"[등분산성] BP p={p_het:.2e} → {'이분산 존재' if p_het < 0.05 else '만족'}")

# 자기상관
dw = durbin_watson(residuals)
print(f"[자기상관] DW={dw:.2f} → {'문제없음' if 1.5 <= dw <= 2.5 else '주의'}")

# 이상치
cooks = OLSInfluence(model).cooks_distance[0]
n_out = (cooks > 4/len(y)).sum()
print(f"[이상치] Cook's D 기준 {n_out}개 ({n_out/len(y)*100:.1f}%)")

[정규성] Shapiro p=7.09e-51 → 불만족(대표본 OK)
[등분산성] BP p=1.68e-217 → 이분산 존재
[자기상관] DW=1.91 → 문제없음
[이상치] Cook's D 기준 334개 (5.5%)


## 6. 사후검정 (Tukey HSD)

In [8]:
tukey = pairwise_tukeyhsd(df['당월_매출_금액'], df['주요_상권유형'], alpha=0.05)
print("[상권유형 간 매출 차이 사후검정]")
print(tukey.summary())

[상권유형 간 매출 차이 사후검정]
             Multiple Comparison of Means - Tukey HSD, FWER=0.05              
group1 group2     meandiff     p-adj       lower            upper       reject
------------------------------------------------------------------------------
  골목상권   관광특구  5770113096.9766    0.0  4898201597.7899  6642024596.1634   True
  골목상권    미분류 -1163054251.8185    0.0 -1622886346.9756  -703222156.6613   True
  골목상권   발달상권   3771815956.391    0.0  3284751487.4614  4258880425.3207   True
  골목상권   전통시장   361650232.9457 0.3799  -189877904.2983   913178370.1897  False
  관광특구    미분류 -6933167348.7951    0.0 -7905533282.5335 -5960801415.0567   True
  관광특구   발달상권 -1998297140.5856    0.0 -2983833416.0478 -1012760865.1233   True
  관광특구   전통시장 -5408462864.0309    0.0 -6427400172.9612 -4389525555.1006   True
   미분류   발달상권  4934870208.2095    0.0   4284872020.847  5584868395.5721   True
   미분류   전통시장  1524704484.7642    0.0   825098071.9486  2224310897.5798   True
  발달상권   전통시장 -3410165723.4453  

## 7. 결과 요약

In [9]:
print("=" * 60)
print("회귀분석 결과 요약")
print("=" * 60)

print(f"""
[모델 적합도]
  R² = {model.rsquared:.4f} (매출 변동의 {model.rsquared*100:.1f}% 설명)
  모델 전체 유의 (F={model.fvalue:.1f}, p<0.001)

[가설 검증 결과]
  H1 유동인구→매출: 채택
    - 100만명 증가 시 {model.params['총_유동인구_수']*1e6/1e8:.2f}억원 증가
    
  H2 점포수→매출: 채택
    - 1개 증가 시 {model.params['점포_수']/1e8:.2f}억원 증가
    
  H3 상권유형→매출: 채택
    - 발달상권: 골목상권 대비 {model.params['발달상권']/1e8:+.2f}억원
    - 관광특구: 골목상권 대비 {model.params['관광특구']/1e8:+.2f}억원
    - 전통시장: 골목상권 대비 {model.params['전통시장']/1e8:+.2f}억원

[모델 진단]
  다중공선성: {max_vif:.1f} ({'양호' if max_vif < 5 else '주의'})
  자기상관: {dw:.2f} ({'양호' if 1.5<=dw<=2.5 else '주의'})

→ 다음: 05_해석.ipynb에서 종합 해석 및 시사점
""")

회귀분석 결과 요약

[모델 적합도]
  R² = 0.6881 (매출 변동의 68.8% 설명)
  모델 전체 유의 (F=2239.0, p<0.001)

[가설 검증 결과]
  H1 유동인구→매출: 채택
    - 100만명 증가 시 1.86억원 증가

  H2 점포수→매출: 채택
    - 1개 증가 시 3.99억원 증가

  H3 상권유형→매출: 채택
    - 발달상권: 골목상권 대비 +14.72억원
    - 관광특구: 골목상권 대비 +27.19억원
    - 전통시장: 골목상권 대비 +11.94억원

[모델 진단]
  다중공선성: 1.5 (양호)
  자기상관: 1.91 (양호)

→ 다음: 05_해석.ipynb에서 종합 해석 및 시사점



In [10]:
# 전체 회귀결과 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               당월_매출_금액   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     2239.
Date:                Tue, 27 Jan 2026   Prob (F-statistic):               0.00
Time:                        15:25:51   Log-Likelihood:            -1.3854e+05
No. Observations:                6097   AIC:                         2.771e+05
Df Residuals:                    6090   BIC:                         2.771e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.732e+08   5.27e+07    -18.469      0.0